In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string
import joblib 

In [8]:
df = pd.read_csv(r"C:\Users\USER\Downloads\FAKE NEWS DETECTION\Fake.csv")
dt = pd.read_csv(r"C:\Users\USER\Downloads\FAKE NEWS DETECTION\True.csv")


In [9]:
df

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [10]:
dt

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [11]:
dt.shape

(21417, 4)

In [12]:
df.shape

(23481, 4)

In [13]:
df.isna().sum

<bound method DataFrame.sum of        title   text  subject   date
0      False  False    False  False
1      False  False    False  False
2      False  False    False  False
3      False  False    False  False
4      False  False    False  False
...      ...    ...      ...    ...
23476  False  False    False  False
23477  False  False    False  False
23478  False  False    False  False
23479  False  False    False  False
23480  False  False    False  False

[23481 rows x 4 columns]>

In [14]:
df.isna().sum

<bound method DataFrame.sum of        title   text  subject   date
0      False  False    False  False
1      False  False    False  False
2      False  False    False  False
3      False  False    False  False
4      False  False    False  False
...      ...    ...      ...    ...
23476  False  False    False  False
23477  False  False    False  False
23478  False  False    False  False
23479  False  False    False  False
23480  False  False    False  False

[23481 rows x 4 columns]>

In [15]:
df["class"]=0

In [16]:
dt["class"]=1

In [17]:
data = pd.concat([df,dt], axis = 0 )

In [18]:
data.sample(10)

,title,text,subject,date,class
19185,Police have thwarted seven attacks since March...,"BRIGHTON, England (Reuters) - London Mayor Sad...",worldnews,"September 24, 2017",1
10767,STUNNING! TERRORISM EXPERT WHO INTERROGATED 9/...,The video below is more important than ever! A...,politics,"May 28, 2017",0
2532,Joy Behar HUMILIATES Trump For Flynn Resignat...,"Donald Trump and his incompetent, corrupt team...",News,"February 14, 2017",0
8276,"Former Canadian PM rebukes Trump on NAFTA, pre...",TORONTO (Reuters) - Former Canadian Prime Mini...,politicsNews,"September 4, 2016",1
11247,Cheesy puns abound as White House hosts online...,(Reuters) - The White House attracted pungent ...,politicsNews,"January 13, 2016",1
13841,Cambodia's Hun Sen calls for closure of rights...,PHNOM PENH (Reuters) - Cambodian Prime Ministe...,worldnews,"November 26, 2017",1
20000,UK transport police leading investigation of L...,LONDON (Reuters) - British counter-terrorism p...,worldnews,"September 15, 2017",1
17129,"U.S., Russia set for likely U.N. row over Syri...",UNITED NATIONS (Reuters) - The United States s...,worldnews,"October 18, 2017",1
15854,Factbox: Gavin Williamson - Britain's new defe...,LONDON (Reuters) - Gavin Williamson was appoin...,worldnews,"November 2, 2017",1
3919,Bill Maher Has A DIRE Election Message The Wh...,We are now down to the wire in the all-importa...,News,"November 6, 2016",0


In [19]:
data = data.drop(["title","subject","date"], axis = 1)

In [20]:
data.reset_index(inplace=True)

In [21]:
data.drop(["index"],axis = 1, inplace=True)

In [22]:
data.sample(5)

,text,class
16492,Nothing says embracing diversity like dividing...,0
15038,Congressman Paul Ryan received enough support ...,0
18986,,0
32632,WASHINGTON (Reuters) - President Barack Obama ...,1
20270,Translation: I m so sorry for your loss but wa...,0


In [23]:
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\[.*?\]', "", text)  # Remove text inside square brackets
    text = re.sub(r'\\W', "", text)  # Remove non-word characters
    text = re.sub(r'https?://\S+|www\.\S+', "", text)  # Remove URLs
    text = re.sub(r'<.*?>', "", text)  # Remove HTML tags
    text = re.sub(r'[%s]' % re.escape(string.punctuation), "", text)  # Remove punctuation
    text = re.sub(r'\n', "", text)  # Remove newline characters
    text = re.sub(r'\w*\d\w*', "", text)  # Remove words containing digits
    return text

In [24]:
data["text"] = data["text"].apply(clean_text)

In [25]:
x=data["text"]
y=data["class"]
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.25,random_state=42)

In [26]:
# Initialize the vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data
xv_train = vectorizer.fit_transform(xtrain)

# Transform the test data
xv_test = vectorizer.transform(xtest)

In [27]:
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression model
lr = LogisticRegression()

# Fit the model on the training data
lr.fit(xv_train, ytrain)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [28]:
prediction = lr.predict(xv_test)
lr.score(xv_test,ytest)

0.985924276169265

In [29]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [31]:
joblib.dump(vectorizer, "vectorizer.jb")
joblib.dump(lr, "lr_model.jb")

['lr_model.jb']